In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

> ⚠️ Kaggle Security Requirement  
> To run this notebook fully (API access + Internet):
> - Log in to your Kaggle account
> - Complete **Phone Verification** in settings
> - Then click **Copy & Edit** 
> - Add a Kaggle Secret named: `GOOGLE_API_KEY`

Without phone verification, you can run only **offline parts**.


# 🌱 Crop Care Agents for Good

This project uses multi-agent intelligence powered by **Gemini Flash** to analyze plant images and provide accurate crop, disease, symptom, and treatment predictions. It works across **color, grayscale, segmented, and healthy plant images**, ensuring robustness across diverse real-world inputs. The system orchestrates multiple agents that collaboratively detect crop type, identify disease, and generate actionable recommendations.

### 🔧 Architecture Overview
- **Image Intake Agent** – Reads any PlantVillage-type image (color / grayscale / segmented).  
- **Crop Detection Agent** – Identifies the correct crop species from the image.  
- **Disease Diagnosis Agent** – Predicts disease or healthy state accurately.  
- **Symptoms & Treatment Agent** – Generates human-level guidance for farmers.  
- **Pipeline Orchestrator** – Connects all agents end-to-end for a complete analysis.




##  How to Run This Notebook (Step-by-Step)

Please execute the notebook **sequentially, one cell at a time**, from top to bottom.
This ensures the multi-agent pipeline runs correctly and all intermediate states are passed properly between agents.

### Setup Instructions

1. **Attach the PlantVillage dataset**
   → From Kaggle Datasets (includes color, grayscale, and segmented leaf images)

2. **Add your Google API Key in Kaggle**

   * Go to: **Settings → Secrets**
   * Create a new secret named: `GOOGLE_API_KEY`
   * Paste your Gemini API key

3. When prompted in the final cell:

   * Enter the full path of any image inside `/kaggle/input/...`

### Output

After the pipeline completes, the final logs will include:

* Crop identification
* Symptom extraction
* Disease classification
* Treatment recommendations



## Install and Import Required Libraries
Ensure PIL and google-generative-ai are installed.


In [ ]:
!pip install -q --upgrade google-generativeai Pillow tqdm
print("✅ Installed/updated dependencies.")

In [ ]:
import json
from PIL import Image
import google.generativeai as genai


## Configure Google Generative AI API Key
Set your API key to authenticate with the Gemini model.


## ⚡ Using Kaggle? Set your Google API Key here

If you are running this notebook on **Kaggle**, **run only this cell** to load your API key from Kaggle Secrets.

Do **not** run the other cells until the API key is properly loaded.

> If you are using Jupyter locally, skip this cell and run the next cell( Follow README to set the `GOOGLE_API_KEY` as an environment variable.)


In [ ]:
#KAGGLE API_KEY SET UP
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Loaded GOOGLE_API_KEY from Kaggle Secrets.")
except Exception as e:
    raise RuntimeError("Add GOOGLE_API_KEY to Kaggle Secrets. Error: " + str(e))


In [ ]:
#JUPYTER_NOTEBOOK API KEY SETUP
import os

os.environ["GOOGLE_API_KEY"] = "YOUR_API_KEY"

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
print("✅ Loaded GOOGLE_API_KEY:", GOOGLE_API_KEY[:8] + "*****")


### 🚀 Why Gemini Flash (Latest)?
I selected **Gemini Flash** because it offers fast, cost-efficient, and highly accurate vision-language reasoning—making it ideal for real-time crop diagnosis and multi-agent processing.


In [ ]:
genai.configure(api_key=GOOGLE_API_KEY)
MODEL = "models/gemini-flash-latest"


## Utility Function: Safe JSON Parsing
This function ensures that JSON returned by the LLM is parsed safely even if extra text is added.


In [ ]:
def safe_json_parse(text):
    try:
        # sometimes the model adds extra text before/after JSON, extract {...}
        start = text.find("{")
        end = text.rfind("}") + 1
        if start == -1 or end == -1:
            return None
        return json.loads(text[start:end])
    except Exception as e:
        print("JSON parsing failed:", e)
        print("Raw output:", text)
        return None


## Agent 1: Crop Detection
This agent detects the crop from the input image and returns JSON with crop name, confidence, and alternatives.


In [ ]:
def detect_crop_with_llm(image_path):
    img = Image.open(image_path)
    prompt = """
You are an agriculture expert.
Identify the crop shown in the image.
Return ONLY JSON like:
{
  "crop": "name of the crop",
  "confidence": "high / medium / low",
  "alternatives": ["possible option1", "option2"]
}
"""
    model = genai.GenerativeModel(MODEL)
    result = model.generate_content([prompt, img])
    return safe_json_parse(result.text)


## Agent 2: Describe Symptoms
This agent generates a textual description of the crop's symptoms from the image.


In [ ]:
def describe_symptoms(crop_name, image_path):
    img = Image.open(image_path)
    prompt = f"""
You are an agricultural expert.
Describe the symptoms visible on the {crop_name} leaf.
Return text only.
"""
    model = genai.GenerativeModel(MODEL)
    result = model.generate_content([prompt, img])
    return result.text.strip()


## PlantVillage Disease Classes  
Using the PlantVillage dataset containing 38 directories (classes) of labeled crop–disease images for classification.


In [ ]:
PLANTVILLAGE_CLASSES = [
    "Apple___Apple_scab",
    "Apple___Black_rot",
    "Apple___Cedar_apple_rust",
    "Apple___healthy",

    "Blueberry___healthy",

    "Cherry_(including_sour)___Powdery_mildew",
    "Cherry_(including_sour)___healthy",

    "Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot",
    "Corn_(maize)___Common_rust_",
    "Corn_(maize)___Northern_Leaf_Blight",
    "Corn_(maize)___healthy",

    "Grape___Black_rot",
    "Grape___Esca_(Black_Measles)",
    "Grape___Leaf_blight_(Isariopsis_Leaf_Spot)",
    "Grape___healthy",
        "Orange___Haunglongbing_(Citrus_greening)",

    "Peach___Bacterial_spot",
    "Peach___healthy",

    "Pepper,_bell___Bacterial_spot",
    "Pepper,_bell___healthy",

    "Potato___Early_blight",
    "Potato___Late_blight",
    "Potato___healthy",

    "Raspberry___healthy",

    "Soybean___healthy",

    "Squash___Powdery_mildew",

    "Strawberry___Leaf_scorch",
    "Strawberry___healthy",
    "Tomato___Bacterial_spot",
    "Tomato___Early_blight",
    "Tomato___Late_blight",
    "Tomato___Leaf_Mold",
    "Tomato___Septoria_leaf_spot",
    "Tomato___Spider_mites Two-spotted_spider_mite",
    "Tomato___Target_Spot",
    "Tomato___Tomato_Yellow_Leaf_Curl_Virus",
    "Tomato___Tomato_mosaic_virus",
    "Tomato___healthy"
]


## Agent 3: Disease Diagnosis
This agent diagnoses the disease from the PlantVillage class list based on the symptoms.


In [ ]:
def diagnose_disease(crop_name, symptoms):
    classes_list = "\n".join(PLANTVILLAGE_CLASSES)
    prompt = f"""
You are an agricultural disease expert.
Crop: {crop_name}
Symptoms: {symptoms}
Diagnose ONLY from this PlantVillage list:
{classes_list}
Return JSON:
{{"disease_name":"...", "reasoning":"...", "confidence":"..." }}
"""
    model = genai.GenerativeModel(MODEL)
    result = model.generate_content([prompt])
    return safe_json_parse(result.text)


## Agent 4: Treatment Recommendation
This agent provides treatment steps and safety precautions for the diagnosed disease.


In [ ]:
def give_treatment(crop_name, disease_name):
    prompt = f"""
You are an agricultural expert.
Provide treatment for {crop_name} affected by {disease_name}.
Return JSON: {{"treatment_steps": ["..."], "safety_precautions": ["..."]}}
"""
    model = genai.GenerativeModel(MODEL)
    result = model.generate_content([prompt])
    return safe_json_parse(result.text)


## Orchestrator: Combine All Agents
This function sequentially calls all agents and merges their outputs into a final structured JSON.


In [ ]:
def orchestrate_pipeline(image_path):
    # Step 1: Detect crop
    crop_info = detect_crop_with_llm(image_path)
    if crop_info is None:
        return {"error": "Failed to detect crop"}

    crop_name = crop_info["crop"]

    # Step 2: Describe symptoms
    symptoms = describe_symptoms(crop_name, image_path)

    # Step 3: Diagnose disease
    disease_info = diagnose_disease(crop_name, symptoms)
    if disease_info is None:
        disease_info = {"disease_name": "Unknown", "reasoning": "Failed LLM diagnosis", "confidence": "low"}

    # Step 4: Provide treatment
    disease_name = disease_info.get("disease_name", "Unknown")
    treatment_info = give_treatment(crop_name, disease_name)
    if treatment_info is None:
        treatment_info = {"treatment_steps": [], "safety_precautions": []}

    # Merge final result
    return {
        "crop_info": crop_info,
        "symptoms": symptoms,
        "disease_info": disease_info,
        "treatment_info": treatment_info
    }


## Dynamic Input and Execution
Enter the full image path inside `/kaggle/input` to run the full multi-agent pipeline.


## 🖼️ Setting the Input Image

You can provide an input leaf image in two ways:

1️⃣ **Kaggle Users**  
- You can use the **PlantVillage dataset** images already available in the input.  
- Or, **upload any image** using the "Add data" option in Kaggle Input.  

2️⃣ **Local Jupyter Notebook**  
- Place your image in the `image/` folder.  
- Enter the image path as:
- Eg :  ../image/COLOR_APPLE_HEALTHY.jpg



In [ ]:
IMAGE_PATH = input("Enter full image path inside /kaggle/input: ").strip()
result = orchestrate_pipeline(IMAGE_PATH)
print(json.dumps(result, indent=2))


### 1. Color Image — Corn (Cercospora Leaf Spot / Gray Leaf Spot)

Input Path:
/kaggle/input/plantvillage-dataset/color/Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot/00a20f6f-e8bd-4453-9e25-36ea70feb626___RS_GLSp 4655.JPG


✔ Model Prediction:

Crop: Corn (Maize)

Disease: Cercospora Leaf Spot (Gray Leaf Spot)

Confidence: High

Symptoms: Narrow, elongated, gray-tan necrotic lesions parallel to veins; chlorotic halos; typical “rectangular” lesions of GLS.

Treatment Summary: Hybrid-resistant varieties, crop rotation, residue management, triazole/strobilurin fungicides timed at V8–VT stage.

In [ ]:
IMAGE_PATH = input("Enter full image path inside /kaggle/input: ").strip()
result = orchestrate_pipeline(IMAGE_PATH)
print(json.dumps(result, indent=2))


### 2. Color Image — Apple (Healthy)

Input Path:
 /kaggle/input/plantvillage-dataset/color/Apple___healthy/0055dd26-23a7-4415-ac61-e0b44ebfaf80___RS_HL 5672.JPG

✔ Model Prediction:

Crop: Apple

Disease: Healthy

Confidence: High

Symptoms: No visible lesions, uniform leaf color, no necrosis, spotting, or fungal growth.

Treatment Summary: None required — general orchard best practices only (irrigation balance, pruning, nutrient management).

In [ ]:
IMAGE_PATH = input("Enter full image path inside /kaggle/input: ").strip()
result = orchestrate_pipeline(IMAGE_PATH)
print(json.dumps(result, indent=2))


### 3. Segmented Image — Apple (Apple Scab)

Input Path:
/kaggle/input/plantvillage-dataset/segmented/Apple___Apple_scab/00075aa8-d81a-4184-8541-b692b78d398a___FREC_Scab 3335_final_masked.jpg

✔ Model Prediction:

Crop: Apple

Disease: Apple Scab

Confidence: High

Symptoms: Dark, necrotic, scab-like lesions on the adaxial leaf surface, irregular in shape and distributed across the lamina.

Treatment Summary: Sanitation of fallen leaves, pruning, preventive fungicide program from Green-Tip to Petal-Fall, summer cover sprays, PHI monitoring, resistance-aware fungicide rotation.

In [ ]:
IMAGE_PATH = input("Enter full image path inside /kaggle/input: ").strip()
result = orchestrate_pipeline(IMAGE_PATH)
print(json.dumps(result, indent=2))


### 4. Grayscale Image — Cherry (Powdery Mildew)

Input Path:
/kaggle/input/plantvillage-dataset/grayscale/Cherry_(including_sour)___Powdery_mildew/00705aa7-5ea2-4419-9440-8ba65e108eb9___FREC_Pwd.M 0267.JPG

✔ Model Prediction:

Crop: Cherry

Disease: Powdery Mildew

Confidence: 0.80

Symptoms: Light-toned lesions, mottling, localized chlorosis/necrosis visible even in grayscale; irregular patches indicating foliar fungal infection.

Treatment Summary: Canopy pruning, nitrogen control, horticultural oils (Neem), potassium bicarbonate, FRAC-rotation fungicides (Myclobutanil, Quinoxyfen, Strobilurins).

## Model Performance Summary

The model successfully handles all image types grayscale, color, and segmented and consistently identifies the correct crop and correct disease from the full PlantVillage class list. It accurately distinguishes healthy leaves from diseased ones and provides precise symptom descriptions along with scientifically valid treatment recommendations. This demonstrates strong generalization and reliability across diverse input variations

**Key Features Implemented**
- ✅ **Multi-agent system** (Crop Detector Agent, Disease Classifier Agent, Symptom & Treatment Agent)  
- ✅ **LLM-powered vision reasoning** using **Gemini Flash (latest)**  
- ✅ **Tool-based workflow** with image loading, JSON parsing, and agent orchestration  
- ✅ **Context engineering** for structured prompts and consistent outputs  
- ✅ **Robust pipeline** that works across RGB, grayscale, healthy, diseased, and segmented leaf images  
